In [ ]:
from Data.Drosophilla.FlyDataMod import FlyDataModule
from Models.Transformer import PositionalEncoding, TransformerModule
from pytorch_lightning.callbacks import EarlyStopping
from IPython.core.debugger import set_trace
import pytorch_lightning as pl
import os
label_types = ["directionality", "directionality", "directionality",
               "insulation","insulation", "insulation"]
label_vals  = [10, 20, 30, 10, 20, 30]

In [ ]:
for i, (label_type, label_val) in enumerate(zip(label_types, label_vals)):
    dm = FlyDataModule(cell_line="S2",
            data_win_radius=4,
            batch_size=1,
            label_type=label_type,
            label_val=label_val)
    dm.setup()
    early_stop_callback = EarlyStopping(
    monitor="val weighted mse loss",
    min_delta=0.00,
    patience=3,
    verbose=False,
    mode='min')
    
    rootdir = "Experiments/Transformer_Other_TAD_Metrics"
    if not os.path.isdir(rootdir):
        os.mkdir(rootdir)

    trainer = pl.Trainer(
        gpus=1,
        max_epochs=200,
        default_root_dir=rootdir,
        callbacks=[early_stop_callback]
    )
    model = TransformerModule(
        ntoken=1,
        ninp=29,
        nhid=2048,
        nhead=1,
        nlayers=5,
        dropout=0.2,
        optimi="Adam",
        lr=0.0001,
        loss_type="mse")
    
    trainer.fit(model, dm)

In [ ]:
from Utils import evaluations as ev
import glob
for i in range(0,6):
    dm = FlyDataModule(cell_line="S2",
            data_win_radius=4,
            batch_size=1,
            label_type=label_types[i],
            label_val=label_vals[i])
    dm.setup()
    layer_weights = glob.glob("Experiments/Transformer_Other_TAD_Metrics/lightning_logs/version_"+str(i)+"/checkpoints/*")[0]
    model         = TransformerModule.load_from_checkpoint(layer_weights).to("cuda:0")
    mets          = ev.getModelMetrics(model,
                  dm,
                  'train')
    ev.createPlot(model,
                  dm,
                 'train',
                 "itfigures")
    print(mets)